## Bioinformatics: Sequence Alignment
Sequence alignment in bioinformatics is the process of comparing DNA, RNA, or protein sequences to find similarities. It helps scientists understand relationships, functions, or evolutionary history. By lining up sequences, we can spot matches, differences, and important regions.

In [63]:
from collections import Counter
import random
import numpy as np

## Bioinformatics II: Sequence Alignment 

References:
Jones and Pevzner 2004, An Introduction to Bioinformatics Algorithms

# Edit distance
Finds how dissimilar two strings are to one another by calculating the minimum number of edits needed to make the two identical. 

# Similarity Searches in bioinformatics
- DNA sequence comparison: when biologists infer a newly sequenced gene’s function by finding similarities with genes of known function. 
- DNA mutation is caused by DNA replication errors leading to substitutions, insertions, and deletions of nucleotides. 

- In 1984, scientists found v-sis oncogene, a cancer-causing gene. They compared it with all known genes from that time, and it matched with a normal growth and development gene. They hypothesized that cancer was caused by the normal growth gene doing its job at the wrong time. 
- Similarly, the Cystic fibrosis gene was discovered through a successful similarity search as well. Cystic fibrosis is a fatal disease which damages body organs and causes thick, lung clogging mucus. 

## Consensus string (dunno if this should be included)
Gets the most common nucleobases per index and then use it in the final string

In [64]:
def consensus_string(matrix):
    consensus = []
    num_columns = len(matrix[0])

    for col in range(num_columns):
        column_bases = [row[col] for row in matrix]   
        counts = Counter(column_bases)                
        most_common = counts.most_common(1)[0][0]     
        consensus.append(most_common)

    return ''.join(consensus)

In [65]:
P = generate_matrix()

for row in P:
    print(" ".join(row))

consensus_string(P)

G G T T C
C T A A C
G A T A T
G T A A C
C T T G A
A A G G T
G A C A G
T C C G C
A C C C A
A A T A G


'GATAC'

## 1. Hamming Distance
The number of positions that differ in two strings of equal lengths. 
The symbols may be letters, bits, or decimal digits, among other possibilities. 
For example, the Hamming distance between:
- "karolin" and "kathrin" is 3.
- "karolin" and "kerstin" is 3.
- "kathrin" and "kerstin" is 4.
- 0000 and 1111 is 4.
- 2173896 and 2233796 is 3.

In [66]:
a = ['A', 'T', 'T', 'G', 'T', 'C']
b = ['A', 'C', 'T', 'C', 'T', 'C']

distance = 0

print(f'Distance: {distance}')

for i in range (len(a)):
  if a[i] != b[i]:
    distance += 1

print(f'Hamming distance: {distance}')

Distance: 0
Hamming distance: 2


In [67]:
# Counts how many nucleobases (A,T,C,G) are different from the REFERENCE SEQUENCE in the first row 

#generate dna sequences of the same length
def generate_dna_sequences_samelength():
    bases = ['A', 'T', 'C', 'G']
    matrix_rows = random.randint(5, 10) #number of dna sequences

    sequences = []
    length = random.randint(5,8)
    for row in range(matrix_rows):
        sequence = [random.choice(bases) for column in range(length)]
        sequences.append(sequence)

    return sequences

X = generate_dna_sequences_samelength()

for row in X:
    print(" ".join(row))

G T C C C A G
C C C C G G T
C T C T C G G
G A G A G G A
C T T G A C C
C C C T C A T


In [68]:
def Hamming_distance(matrix, reference_index=0):
    reference = matrix[reference_index] #compare to the first row
    distance = 0
    for row in matrix:
        for i in range(len(reference)):
            if row[i] != reference[i]:
                distance += 1
    return distance

Hamming_distance(X)

24

However, it only works with sequences of identical lengths

In [69]:
def generate_random_dna_sequences():
    bases = ['A', 'T', 'C', 'G']
    num_sequences = random.randint(5, 10)

    sequences = []
    for _ in range(num_sequences):
        length = random.randint(5, 15)
        sequence = [random.choice(bases) for _ in range(length)]
        sequences.append(sequence)

    return sequences

random_dna = generate_random_dna_sequences()
for i, seq in enumerate(random_dna, 1):
    print(seq)

X = generate_random_dna_sequences()
Hamming_distance(X)


['A', 'T', 'G', 'G', 'T', 'T', 'C', 'A', 'A', 'A', 'C', 'G', 'G', 'T']
['G', 'G', 'A', 'C', 'T', 'T', 'A']
['C', 'T', 'C', 'T', 'T', 'A', 'T', 'A', 'T', 'A', 'T', 'A', 'C']
['G', 'C', 'T', 'C', 'T', 'C', 'C', 'G', 'A', 'T', 'C', 'G', 'G']
['G', 'G', 'T', 'T', 'G', 'C', 'G', 'T', 'T', 'G', 'C']
['G', 'A', 'C', 'A', 'T', 'G', 'G', 'T', 'C', 'G', 'A', 'C', 'T', 'C', 'G']
['A', 'G', 'G', 'G', 'G', 'G', 'T', 'G', 'A', 'C', 'C', 'A', 'C', 'T', 'G']
['A', 'A', 'C', 'G', 'C', 'A', 'A', 'A', 'C', 'A', 'T']


20

## 2. Levenshtein 
The levevnshtein edit distance works for strings with different lengths

In [70]:
def levenshtein(string_one, string_two):
    m = len(string_one)
    n = len(string_two)

    '''
    if m == 0:
        return n
    if n == 0:
        return m
    '''

    dp = [[0]*(n + 1) for _ in range(m + 1)]

    for i in range(m + 1):
        dp[i][0] = i
    for j in range(n + 1):
        dp[0][j] = j

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if string_one[i - 1] == string_two[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = 1 + min([
                                    dp[i][j - 1], 
                                    dp[i - 1][j], 
                                    dp[i - 1][j - 1]
                                    ])
    return dp[m][n]

In [71]:
L = generate_random_dna_sequences()

for i, seq in enumerate(L, 1):
    print(seq)

n = len(L)
dist_matrix = np.zeros((n, n), dtype=int)

for i in range(n):
    for j in range(n):
        dist_matrix[i, j] = levenshtein(L[i], L[j])

# Display the distance matrix
print("\nLevenshtein Distance Matrix:")
for row in dist_matrix:
    print(' '.join(map(str, row)))


['T', 'G', 'C', 'G', 'G']
['G', 'A', 'G', 'T', 'C', 'G', 'C', 'A']
['G', 'T', 'A', 'G', 'T', 'A', 'G', 'A', 'T', 'T', 'A', 'C', 'C', 'A']
['T', 'A', 'C', 'C', 'G', 'A', 'A', 'C', 'C', 'A', 'G', 'G', 'C', 'T']
['G', 'G', 'C', 'C', 'G', 'A', 'T', 'T', 'A', 'C', 'A', 'G', 'C']
['T', 'C', 'C', 'G', 'C', 'G', 'G', 'C', 'C', 'G', 'C']

Levenshtein Distance Matrix:
0 5 11 9 9 6
5 0 7 9 8 7
11 7 0 10 7 11
9 9 10 0 7 7
9 8 7 7 0 7
6 7 11 7 7 0


## 3. Multisequence
Description

## 4. Local
Regular and affine

## 5. Global
Regular and affine

## 6. BLAST 
Why blast is fast & 
Why blast is incomplete 